<center><font size = 6><b>Generate Fake Data & Load To MySQL DB</b></font></center>

### Module

In [1]:
# !pip install Faker
# !pip install pymysql

In [2]:
import re
import faker
import pymysql
import pandas as pd

from faker import Faker
fake = Faker()

print(f"Pandas Version :- {pd.__version__}")
print(f"Faker Version :- {faker.VERSION}")
print(f"PyMySQL Version :- {pymysql.__version__}")

Pandas Version :- 1.5.3
Faker Version :- 19.2.0
PyMySQL Version :- 1.4.6


### Required Inputs

In [3]:
# Database Credentials

username = 'root'
password = 'MySql_1234'
host = 'localhost'
port = 3306
DB_NAME = 'store'

In [4]:
# Database Queries
DB_Schema= "company"
DB_Table= "employee_details"

DB_Queries = [
f"CREATE DATABASE IF NOT EXISTS {DB_Schema};",
f"USE {DB_Schema};",
f"""CREATE TABLE IF NOT EXISTS {DB_Schema}.{DB_Table}(
row_no INT, emp_id INT PRIMARY KEY, 
first_name VARCHAR(100), last_name VARCHAR(100), 
phone BIGINT, email VARCHAR(100), 
primary_skills VARCHAR(100), experience INT, 
salary INT, gender VARCHAR(100),
street_addr VARCHAR(100), city VARCHAR(100),
state VARCHAR(100), zipcode INT  );"""
]

Insert_Query = f"""INSERT INTO {DB_Schema}.{DB_Table} (
row_no,emp_id,first_name,last_name,phone,email,primary_skills,
experience, salary,gender,street_addr,city,state,zipcode)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

In [5]:
# Inputs For Fake Data Generation

Gender = ["Male","Female"]
No_Of_EMP = 50
Skills  = ['Python', 'HTML', 'JavaScript',
               'CSS', 'PHP', 'SQL', 'C++', 'Ruby', 'SQL']

### UDF

In [6]:
# This will generate name as per gender
def Name_Generator(gender):
    if gender == "Male":
        return fake.first_name_male()
    else:
        return fake.first_name_female()

# This will generate phone number    
def Ph_No_Generator():
    ph = fake.basic_phone_number()
    # Removing everything which is not number
    ph = int(re.sub(r'[^0-9]',r'', ph))
    
    return ph

# MySQL DB Connection & Cursor
def Create_DB_Connection(username,password,host,port):
    return pymysql.connect(user = username,
                           password = password,
                           host = host,
                           port = port)

### Create DB Connection

In [7]:
Conn = Create_DB_Connection(username,password,host,port)
Conn.autocommit = False
Cursor = Conn.cursor()

### Create Schema & Table

In [8]:
try:
    for each_query in DB_Queries:  
        Cursor.execute(each_query)
        Conn.commit()
    
except Exception as e:
    Conn.rollback()
    print(e)

### Generate Fake Data & Load To MySQL DB

In [9]:
Row_No = 0
try:
    for each_gender in Gender:
        for _ in range(No_Of_EMP):
            row_no = Row_No + 1,
            emp_id = fake.unique.random_int(min=1001, max=99999999),
            first_name = Name_Generator(gender = each_gender),
            last_name = fake.last_name(),
            phone = Ph_No_Generator(),
            email = fake.email(),
            primary_skills = fake.random_element(elements=(Skills)),
            experience = fake.random_int(min=0, max=35),
            salary = fake.random_int(min=50000, max=100000, step=1000),
            gender = each_gender,
            street_addr = fake.street_address(),
            city = fake.city(),
            state = fake.state_abbr(),
            zipcode = (fake.zipcode())

            Cursor.execute(Insert_Query,(row_no,emp_id,first_name,last_name,
                                 phone,email,primary_skills,experience,
                                 salary,gender,street_addr,city,state,zipcode))
            Row_No += 1
            Conn.commit()
            
except Exception as e:
    Conn.rollback()
    print(e)

In [10]:
# Conn.close()
# Cursor.close()